# **Training an Testing**

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

In [14]:
def load_data_files():

    base_dir = os.path.abspath(os.path.dirname(__file__)) if '__file__' in globals() else os.getcwd()

    data_dir = os.path.join(base_dir, 'Processed Data')

    X_train_path = os.path.join(data_dir, 'X_train_processed.npy')
    y_train_path = os.path.join(data_dir, 'y_train_processed.npy')
    X_test_path = os.path.join(data_dir, 'X_test_processed.npy')
    y_test_path = os.path.join(data_dir, 'y_test_processed.npy')

    X_train_mx = np.load(X_train_path, allow_pickle = True).item()
    y_train_mx = np.load(y_train_path, allow_pickle = True)
    X_test_mx = np.load(X_test_path, allow_pickle = True).item()
    y_test_mx = np.load(y_test_path, allow_pickle = True)

    X_train = pd.DataFrame(X_train_mx.toarray())
    y_train = pd.Series(y_train_mx.ravel())
    X_test = pd.DataFrame(X_test_mx.toarray())
    y_test = pd.Series(y_test_mx.ravel())

    return X_train, y_train, X_test, y_test

In [15]:
# Call the function

X_train, y_train, X_test, y_test = load_data_files()

In [16]:
def load_column_names():

    base_dir = os.path.abspath(os.path.dirname(__file__)) if '__file__' in globals() else os.getcwd()

    data_dir = os.path.join(base_dir, 'Processing Objects')

    column_names_path = os.path.join(data_dir, 'the_preprocessor.joblib')

    column_names_pre = joblib.load(column_names_path)

    column_names = column_names_pre.get_feature_names_out()

    return column_names

In [17]:
# Call the function

column_names = load_column_names()

In [18]:
# Match the name with the corresponding column

X_train.columns = column_names
X_test.columns = column_names

In [19]:
print(column_names.tolist())

['numerical__Age', 'numerical__EducationLevel', 'numerical__BMI', 'numerical__CognitiveTestScore', 'binary_categorical__Gender_Male', 'binary_categorical__Diabetes_Yes', 'binary_categorical__Hypertension_Yes', 'binary_categorical__CholesterolLevel_Normal', 'binary_categorical__AlzheimerInFamily_Yes', 'binary_categorical__GeneticRisk_Yes', 'binary_categorical__UrbanRural_Urban', 'multi_binary_categorical__Country_Argentina', 'multi_binary_categorical__Country_Australia', 'multi_binary_categorical__Country_Brazil', 'multi_binary_categorical__Country_Canada', 'multi_binary_categorical__Country_China', 'multi_binary_categorical__Country_France', 'multi_binary_categorical__Country_Germany', 'multi_binary_categorical__Country_India', 'multi_binary_categorical__Country_Italy', 'multi_binary_categorical__Country_Japan', 'multi_binary_categorical__Country_Mexico', 'multi_binary_categorical__Country_Norway', 'multi_binary_categorical__Country_Russia', 'multi_binary_categorical__Country_Saudi Ara

# **Hyperparameter Tuning**

In [20]:
import optuna
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score

In [21]:
# Stratified Cross Validation

cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

In [22]:
# Dictionary to save the results

model_results = {}

In [23]:
# Hyperparameter Tuning with Optuna (Objective Functions)



# XGBoost

# Personalized Wrapping, to ensure that XGBoost Classifier behaves like a Sklearn method

from sklearn.base import BaseEstimator, ClassifierMixin
from xgboost import XGBClassifier

class SklearnXGBClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, **kwargs):

        self.model = XGBClassifier(**kwargs)

    def fit(self, X, y):

        self.model.fit(X, y)

        return self

    def predict(self, X):
        
        return self.model.predict(X)

    def predict_proba(self, X):
        
        return self.model.predict_proba(X)

    def get_params(self, deep = True):
        
        return self.model.get_params(deep)

    def set_params(self, **params):
        
        self.model.set_params(**params)
        
        return self



def xgb_objective(trial):
    
    model = SklearnXGBClassifier(
        n_estimators = trial.suggest_int('n_estimators', 50, 150),
        max_depth = trial.suggest_int('max_depth', 5, 15),
        learning_rate = trial.suggest_float('learning_rate', 1e-2, 1e-1, log = True),
        subsample = trial.suggest_float('subsample', 0.5, 1.0),
        eval_metric = 'logloss',random_state = 42,n_jobs = -1
    )

    return cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy').mean()



# Stacking

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

def stacking_objective(trial):

    estimators = [
        ('rf', RandomForestClassifier(n_estimators = 50, random_state = 42)),
        ('dt', DecisionTreeClassifier(min_samples_leaf = 2, random_state = 42)),
        ('gbc', GradientBoostingClassifier(n_estimators = 50, random_state = 42))
    ]

    meta_estimator = LogisticRegression(
        C = trial.suggest_float('C', 1e-2, 10.0, log = True),
        max_iter = 1000,
        random_state = 42
    )

    model = StackingClassifier(
        estimators = estimators,
        final_estimator = meta_estimator,
        passthrough = trial.suggest_categorical('passthrough', [True, False]),
        n_jobs = -1
    )

    return cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy').mean()



# Random Forest

from sklearn.ensemble import RandomForestClassifier

def rf_objective(trial):

    model = RandomForestClassifier(
        n_estimators = trial.suggest_int('n_estimators', 50, 100),
        max_depth = trial.suggest_int('max_depth', 15, 50),
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10),
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10),
        random_state = 42, n_jobs = -1
    )

    return cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy').mean()


# LightGBM

from lightgbm import LGBMClassifier

def lgbm_objective(trial):
    
    model = LGBMClassifier(
        n_estimators = trial.suggest_int('n_estimators', 50, 150),
        max_depth = trial.suggest_int('max_depth', 3, 15),
        learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1, log = True),
        num_leaves = trial.suggest_int('num_leaves', 20, 100),
        subsample = trial.suggest_float('subsample', 0.5, 1.0),
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0),
        random_state = 42, n_jobs=-1, verbosity = -1
    )

    return cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy').mean()



# KNN

from sklearn.neighbors import KNeighborsClassifier

def knn_objective(trial):

    model = KNeighborsClassifier(
        n_neighbors = trial.suggest_int('n_neighbors', 3, 8),
        metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
    )

    return cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy').mean()


# Decision Tree

from sklearn.tree import DecisionTreeClassifier

def dt_objective(trial):

    model = DecisionTreeClassifier(
        max_depth = trial.suggest_int('max_depth', 3, 12),
        min_samples_split = trial.suggest_int('min_samples_split', 2, 7),
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 7),
        criterion = trial.suggest_categorical('criterion', ['gini', 'entropy']), 
        random_state = 42
    )

    return cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy').mean()


# Multi-Layer Perceptron

from sklearn.neural_network import MLPClassifier

def mlp_objective(trial):

    model = MLPClassifier(
        hidden_layer_sizes = (trial.suggest_int('hl1', 10, 40),
                              trial.suggest_int('hl2', 10, 50)),
        activation = trial.suggest_categorical('activation', ['relu', 'tanh']),
        alpha = trial.suggest_float('alpha', 1e-3, 1e-1, log = True),
        learning_rate = trial.suggest_categorical('learning_rate', ['constant', 'adaptive']),
        learning_rate_init = trial.suggest_float('learning_rate_init', 1e-4, 1e-1, log = True),
        max_iter = 500, random_state = 42
        
    )

    return cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy').mean()


# Gradient Boost Classifier

from sklearn.ensemble import GradientBoostingClassifier

def gbc_objective(trial):
    
    model = GradientBoostingClassifier(
        n_estimators = trial.suggest_int('n_estimators', 50, 150),
        learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1, log = True),
        max_depth = trial.suggest_int('max_depth', 3, 10),
        subsample = trial.suggest_float('subsample', 0.5, 1.0),
        random_state = 42
    )

    return cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy').mean()



# Support Vector Machine (SVM)

from sklearn.svm import SVC

def svm_objective(trial):

    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'sigmoid'])
    gamma = trial.suggest_float('gamma', 1e-4, 1e-1, log = True)
    model = SVC(kernel = kernel, gamma = gamma)

    return cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy').mean()


In [24]:
# Execute the tuning and save the models as tuples

models = {
    'LightGBM': (lgbm_objective, LGBMClassifier),
    'XGBoost': (xgb_objective, XGBClassifier),
    'Stacking': (stacking_objective, StackingClassifier),
    'Random Forest': (rf_objective, RandomForestClassifier),
    'KNN': (knn_objective, KNeighborsClassifier),
    'Decision Tree': (dt_objective, DecisionTreeClassifier),
    'MLP': (mlp_objective, MLPClassifier),
    'Gradient Boosting': (gbc_objective, GradientBoostingClassifier),
    'SVM': (svm_objective, SVC),
}


for name, (objective, constructor) in models.items():

    print("Optimizating", name, " ...")
    print("------------------------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------------------------")

    study = optuna.create_study(direction = 'maximize')
    study.optimize(objective, n_trials = 10)

    print("Best accuracy: ", study.best_value)
    print("Hyperparameters: ", study.best_params)

    
    # Train the model with the best hyperparameter fit

    if name == 'MLP':
        params = study.best_params.copy()
        hl1 = params.pop('hl1')
        hl2 = params.pop('hl2')
        final_model = constructor(
            hidden_layer_sizes = (hl1, hl2),
            **params
        )

    elif name == 'Stacking':
        params = study.best_params.copy()
        passthrough = params.pop('passthrough')
        C = params.pop('C')

        estimators = [
            ('rf', RandomForestClassifier(n_estimators = 50, random_state = 42)),
            ('dt', DecisionTreeClassifier(min_samples_leaf = 2, random_state = 42)),
            ('gbc', GradientBoostingClassifier(n_estimators = 50, random_state = 42))
        ]

        final_model = StackingClassifier(
            estimators = estimators,
            final_estimator = LogisticRegression(C = C, max_iter = 1000, random_state = 42),
            passthrough = passthrough, n_jobs = -1
        )

    else:
        final_model = constructor(**study.best_params)


    # Fit the model

    final_model.fit(X_train, y_train)

    # Evaluate on test set

    y_pred = final_model.predict(X_test)
    acc_metric = accuracy_score(y_test, y_pred)

    print("Accuracy in test: ", acc_metric)

    # Save results

    model_results[name] = {
        'accuracy_test': acc_metric,
        'model': final_model,
        'parameters': study.best_params,
        'study': study
    }

[I 2025-06-17 11:46:13,148] A new study created in memory with name: no-name-a2746b18-ed60-4aa5-a0a7-8197eba6c2b1


Optimizating LightGBM  ...
------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------


[I 2025-06-17 11:46:16,415] Trial 0 finished with value: 0.7072194663688076 and parameters: {'n_estimators': 116, 'max_depth': 13, 'learning_rate': 0.010643877203710038, 'num_leaves': 72, 'subsample': 0.7066337035847874, 'colsample_bytree': 0.7031257684047441}. Best is trial 0 with value: 0.7072194663688076.
[I 2025-06-17 11:46:18,079] Trial 1 finished with value: 0.7204892371307892 and parameters: {'n_estimators': 131, 'max_depth': 5, 'learning_rate': 0.030128576176060066, 'num_leaves': 58, 'subsample': 0.949786071242218, 'colsample_bytree': 0.8922775142023589}. Best is trial 1 with value: 0.7204892371307892.
[I 2025-06-17 11:46:18,773] Trial 2 finished with value: 0.6076387781369511 and parameters: {'n_estimators': 80, 'max_depth': 3, 'learning_rate': 0.0031133302428775636, 'num_leaves': 26, 'subsample': 0.6532210406120806, 'colsample_bytree': 0.9768889730661925}. Best is trial 1 with value: 0.7204892371307892.
[I 2025-06-17 11:46:19,608] Trial 3 finished with value: 0.72179698121860

Best accuracy:  0.7217969812186084
Hyperparameters:  {'n_estimators': 131, 'max_depth': 3, 'learning_rate': 0.09867066541086178, 'num_leaves': 44, 'subsample': 0.5329032825641946, 'colsample_bytree': 0.5109418476113343}


[I 2025-06-17 11:46:26,658] A new study created in memory with name: no-name-27a01f9e-aa43-4447-aeb5-eb2971dd913b


...

Best accuracy:  0.7203161472626807
Hyperparameters:  {'n_estimators': 133, 'learning_rate': 0.017722416290695236, 'max_depth': 3, 'subsample': 0.8710330565191053}


[I 2025-06-17 12:34:14,844] A new study created in memory with name: no-name-0a592a49-b16c-4c31-8f3e-fa1497a19559


Accuracy in test:  0.7199461521202603
Optimizating SVM  ...
------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------


[I 2025-06-17 13:03:38,261] Trial 0 finished with value: 0.7126428095896794 and parameters: {'kernel': 'linear', 'gamma': 0.0013352444014453847}. Best is trial 0 with value: 0.7126428095896794.
[I 2025-06-17 13:33:50,011] Trial 1 finished with value: 0.7126428095896794 and parameters: {'kernel': 'linear', 'gamma': 0.0928752276724382}. Best is trial 0 with value: 0.7126428095896794.
[I 2025-06-17 13:45:18,129] Trial 2 finished with value: 0.7092003890906671 and parameters: {'kernel': 'rbf', 'gamma': 0.01799727470933265}. Best is trial 0 with value: 0.7126428095896794.
[I 2025-06-17 13:56:46,519] Trial 3 finished with value: 0.713969804788922 and parameters: {'kernel': 'rbf', 'gamma': 0.0020622454773098968}. Best is trial 3 with value: 0.713969804788922.
[I 2025-06-17 14:08:17,589] Trial 4 finished with value: 0.7087003983371183 and parameters: {'kernel': 'rbf', 'gamma': 0.0209961106581337}. Best is trial 3 with value: 0.713969804788922.
[I 2025-06-17 14:19:39,960] Trial 5 finished with 

Best accuracy:  0.713969804788922
Hyperparameters:  {'kernel': 'rbf', 'gamma': 0.0020622454773098968}
Accuracy in test:  0.7128113080547454


In [35]:
# Find the best parameters for each model

print("\nResumen de precisión en test para cada modelo:\n")

for name, result in model_results.items():
    accuracy = round(result['accuracy_test'], 4)
    print(name + ": Accuracy = " + str(accuracy))


Resumen de precisión en test para cada modelo:

LightGBM: Accuracy = 0.7221
XGBoost: Accuracy = 0.7208
Stacking: Accuracy = 0.7197
Random Forest: Accuracy = 0.7186
KNN: Accuracy = 0.6538
Decision Tree: Accuracy = 0.7199
MLP: Accuracy = 0.7152
Gradient Boosting: Accuracy = 0.7199
SVM: Accuracy = 0.7128
